In [1]:
import os
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_learning_rate: float
    params_image_size: list
    
    
    

In [5]:
from pixel_predictor.constants import *
from pixel_predictor.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
                 ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
          config = self.config.prepare_base_model

          create_directories([config.root_dir])

          prepare_base_model_config = PrepareBaseModelConfig(
                root_dir = Path(config.root_dir),
                base_model_path= Path(config.base_model_path),
                params_image_size= self.params.IMAGE_SIZE,
                params_learning_rate= self.params.LEARNING_RATE,
                
          )
          return prepare_base_model_config

In [9]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense


In [12]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    def get_base_model(self):
        
        self.model = Sequential([
            Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=self.config.params_image_size),
            Conv2D(64, kernel_size=(3, 3), activation='relu'),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(2)
        ])

        self.save_model(path=self.config.base_model_path, model=self.model)
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    

In [13]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2024-05-21 16:40:31,640: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-21 16:40:31,643: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-21 16:40:31,644: INFO: common: created directory at: artifacts]
[2024-05-21 16:40:31,645: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-05-21 16:40:31,719: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
